## Загрузка данных и подключение библиотек

In [1]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c tabular-playground-series-may-2021
!pip install catboost
!pip install eli5
!pip install optuna
!pip install shap
!pip install scikit-learn-extra
!unzip /content/test.csv.zip
!unzip /content/train.csv.zip
!unzip /content/sample_submission.csv.zip

kaggle.json
  0% 0.00/1.72M [00:00<?, ?B/s]
100% 1.72M/1.72M [00:00<00:00, 119MB/s]
  0% 0.00/851k [00:00<?, ?B/s]
100% 851k/851k [00:00<00:00, 115MB/s]
  0% 0.00/128k [00:00<?, ?B/s]
100% 128k/128k [00:00<00:00, 108MB/s]
     |████████████████████████████████| 67.3MB 45kB/s 
     |████████████████████████████████| 112kB 7.6MB/s 
     |████████████████████████████████| 296kB 10.9MB/s 
     |████████████████████████████████| 174kB 51.9MB/s 
     |████████████████████████████████| 81kB 12.1MB/s 
     |████████████████████████████████| 81kB 12.7MB/s 
     |████████████████████████████████| 143kB 57.1MB/s 
     |████████████████████████████████| 112kB 52.6MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11107 sha256=7ae32aa6d0d8757e84e28b56991e537856f7cdb4bea06ad87dc2d779cdff9895
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully bui

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import catboost as cb
import sklearn
import xgboost as xgb
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.decomposition import PCA, FactorAnalysis as FA
from typing import List, Optional
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold, train_test_split
from typing import List, Tuple
import scipy.stats as ss
from sklearn_extra.cluster import KMedoids
import math
from sklearn.utils.validation import check_is_fitted
import eli5
from sklearn.base import BaseEstimator, TransformerMixin
import time
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from sklearn.inspection import permutation_importance
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import shap
import missingno as msno
from sklearn.inspection import permutation_importance
from eli5.sklearn import PermutationImportance
import optuna
from sklearn.metrics import log_loss
pd.plotting.register_matplotlib_converters()
%matplotlib inline
sns.set(color_codes=True)
pal = sns.color_palette("viridis", 10)
sns.set_palette(pal)

## Используемые функции

In [2]:
def get_input(data_path: str) -> pd.DataFrame:
  """
  Считывание данных и вывод основной информации о наборе данных.

  Parmeters
  ---------
  data_path: str - название файла

  Returns
  -------
  data: pandas.core.frame.DataFrame - загруженный набор данных в pandas.Dataframe
  """
  base_path = "/content"
  data = pd.read_csv(f"{base_path}/{data_path}")
  data.columns = [col.lower() for col in data.columns]
  print(f"{data_path}: shape = {data.shape[0]} rows, {data.shape[1]} cols")
  return data

In [3]:
def plot_feature_importance(importance, names, model_type, figsize=(10,8)):

  #Create arrays from feature importance and feature names
  feature_importance = np.array(importance)
  feature_names = np.array(names)

  #Create a DataFrame using a Dictionary
  data={'feature_names':feature_names,'feature_importance':feature_importance}
  fi_df = pd.DataFrame(data)

  #Sort the DataFrame in order decreasing feature importance
  fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

  #Define size of bar plot
  plt.figure(figsize=figsize)
  #Plot Searborn bar chart
  sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
  #Add chart labels
  plt.title(model_type + ' FEATURE IMPORTANCE')
  plt.xlabel('FEATURE IMPORTANCE')
  plt.ylabel('FEATURE NAMES')
  return list(fi_df['feature_names'].values)

In [4]:
def multi_estimators_predict(estimators: List,
                             x_valid: pd.DataFrame,
                             y_valid = None,
                             metric: callable = None,
                             scalers: List = None):
  preds = []
  evals = []
  if scalers:
    assert len(estimators) == len(scalers)

  for i in range(len(estimators)):
    if scalers:
      pred = estimators[i].predict_proba(scalers[i].transform(x_valid))
    else:
      if type(estimators[0]) == xgb.core.Booster:
        pred = estimators[i].predict(x_valid)
      else:
        pred = estimators[i].predict_proba(x_valid)

    
    preds.append(pred)
    if (y_valid is not None) and (metric is not None):
      eval = metric(y_valid, pred)
      evals.append(eval)
  
  result = np.stack([preds], axis=0)
  result = np.squeeze(result, axis=0)
  result = np.mean(result, axis=0)

  if (y_valid is not None) & (metric is not None):
    for i, eval in enumerate(evals):
      print(f"Model {i} metric: {eval:.7}")
    print(f"Result model metric: {metric(y_valid, result):.7}")

  return result

In [5]:
def logistic_cv_fit(params, X, y, cv, scale=False):
    """
    Кросс-валидация для модели LogisticRegression.

    Parameters
    ----------
    params: dict
        Словарь гиперпараметров модели.

    X: pandas.core.frame.DataFrame
        Матрица признаков для обучения модели.

    y: pandas.core.frame.Series
        Вектор целевой переменной для обучения модели.

    cv: KFold or StratifiedKFold generator.
        Объект KFold / StratifiedKFold для определения
        стратегии кросс-валидации модели.

    Returns
    -------
    estimators: list
        Список с объектами обученной модели.

    oof_preds: np.array
        Вектор OOF-прогнозов.

    """

    estimators, folds_scores, scalers = [], [], []
    oof_preds = np.zeros((X.shape[0], 4))
    print(f"{time.ctime()}, Cross-Validation, {X.shape[0]} rows, {X.shape[1]} cols")

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]
        if scale:
          scaler = StandardScaler()
          x_train = scaler.fit_transform(x_train)
          x_valid = scaler.transform(x_valid)
        model = LogisticRegression(**params)
        model.fit(x_train, y_train)
        oof_preds[valid_idx] = model.predict_proba(x_valid)
        score = metrics.log_loss(y_valid, oof_preds[valid_idx])
        print(f"Fold {fold+1}, Valid score = {round(score, 5)}")
        folds_scores.append(round(score, 5))
        estimators.append(model)
        if scale:
          scalers.append(scaler)

    print(f"Score by each fold: {folds_scores}")
    print("="*65)
    return estimators, oof_preds, scalers

In [6]:
def catboost_cv_fit(params, X, y, cv, categorical = None):
    """
    Кросс-валидация для модели catboost.

    Parameters
    ----------
    params: dict
        Словарь гиперпараметров модели.

    X: pandas.core.frame.DataFrame
        Матрица признако для обучения модели.

    y: pandas.core.frame.Series
        Вектор целевой переменной для обучения модели.

    cv: KFold or StratifiedKFold generator.
        Объект KFold / StratifiedKFold для определения
        стратегии кросс-валидации модели.

    categorical: str, optional, default = None
        Список категориальных признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    estimators: list
        Список с объектами обученной модели.

    oof_preds: np.array
        Вектор OOF-прогнозов.

    """
    if not categorical:
        categorical = "auto"

    estimators, folds_scores = [], []
    oof_preds = np.zeros((X.shape[0], 4))
    print(f"{time.ctime()}, Cross-Validation, {X.shape[0]} rows, {X.shape[1]} cols")

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model = cb.CatBoostClassifier(**params)
        model.fit(
            x_train, y_train,
            #eval_set=[(x_valid, y_valid)], 
            verbose=10, 
            #early_stopping_rounds=100,
            #cat_features=categorical
        )
        oof_preds[valid_idx] = model.predict_proba(x_valid)
        score = metrics.log_loss(y_valid, oof_preds[valid_idx])
        print(f"Fold {fold+1}, Valid score = {round(score, 5)}")
        folds_scores.append(round(score, 5))
        estimators.append(model)

    print(f"Score by each fold: {folds_scores}")
    print("="*65)
    return estimators, oof_preds

In [7]:
def lightgbm_cv_fit(params, X, y, cv, categorical = None):
    """
    Кросс-валидация для модели lightgbm.

    Parameters
    ----------
    params: dict
        Словарь гиперпараметров модели.

    X: pandas.core.frame.DataFrame
        Матрица признако для обучения модели.

    y: pandas.core.frame.Series
        Вектор целевой переменной для обучения модели.

    cv: KFold or StratifiedKFold generator.
        Объект KFold / StratifiedKFold для определения
        стратегии кросс-валидации модели.

    categorical: str, optional, default = None
        Список категориальных признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    estimators: list
        Список с объектами обученной модели.

    oof_preds: np.array
        Вектор OOF-прогнозов.

    """
    if not categorical:
        categorical = "auto"

    estimators, folds_scores = [], []
    oof_preds = np.zeros((X.shape[0], 4))
    print(f"{time.ctime()}, Cross-Validation, {X.shape[0]} rows, {X.shape[1]} cols")

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model = lgb.LGBMClassifier(**params)
        model.fit(
            x_train, y_train,
            #eval_set=[(x_valid, y_valid)],
            #eval_metric="multi_logloss", 
            verbose=10, 
            #early_stopping_rounds=50,
            #categorical_feature=categorical
        )
        oof_preds[valid_idx] = model.predict_proba(x_valid)
        score = metrics.log_loss(y_valid, oof_preds[valid_idx])
        print(f"Fold {fold+1}, Valid score = {round(score, 5)}")
        folds_scores.append(round(score, 5))
        estimators.append(model)

    print(f"Score by each fold: {folds_scores}")
    print("="*65)
    return estimators, oof_preds

In [8]:
def xgboost_cv_fit(params, X, y, cv, categorical = None):
    """
    Кросс-валидация для модели xgboost.

    Parameters
    ----------
    params: dict
        Словарь гиперпараметров модели.

    X: pandas.core.frame.DataFrame
        Матрица признако для обучения модели.

    y: pandas.core.frame.Series
        Вектор целевой переменной для обучения модели.

    cv: KFold or StratifiedKFold generator.
        Объект KFold / StratifiedKFold для определения
        стратегии кросс-валидации модели.

    categorical: str, optional, default = None
        Список категориальных признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    estimators: list
        Список с объектами обученной модели.

    encoders: dict
        Список с объектами LabelEncoders.

    oof_preds: np.array
        Вектор OOF-прогнозов.

    """
    estimators, encoders = [], {}
    oof_preds = np.zeros((X.shape[0], 4))

    if categorical:
        for feature in categorical:
            encoder = LabelEncoder()
            X[feature] = encoder.fit_transform(X[feature].astype("str").fillna("NA"))
            encoders[feature] = encoder

    print(f"{time.ctime()}, Cross-Validation, {X.shape[0]} rows, {X.shape[1]} cols")

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):

        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model = xgb.XGBClassifier(**params)
        model.fit(
            x_train, y_train,
            #eval_set=[(x_valid, y_valid)],
            #eval_metric="mlogloss", 
            verbose=10, 
            #early_stopping_rounds=50,
        )

        oof_preds[valid_idx] = model.predict_proba(x_valid)
        score = metrics.log_loss(y_valid, oof_preds[valid_idx])
        print(f"Fold {fold+1}, Valid score = {round(score, 5)}")
        estimators.append(model)

    return estimators, encoders, oof_preds

## Загрузка данных и построение моделей

### lightgbm + optuna

In [166]:
def objective(trial):
    data = get_input("train.csv")
    data.drop(columns='id', inplace=True)
    mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
    data['target'] = data['target'].map(mapper)
    target = data['target']
    data = data.drop(columns=['target'])

    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.5)
    dtrain = lgb.Dataset(train_x, label=train_y)
    dvalid = lgb.Dataset(valid_x, label=valid_y)

    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        #"n_estimators": trial.suggest_int("n_estimators", 200, 800),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        'num_class': 4,
        'max_depth': trial.suggest_int('max_depth', 3,7),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.05, 0.13),
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "multi_logloss")
    gbm = lgb.train(
        param, dtrain, 
        valid_sets=[dvalid], 
        verbose_eval=False, 
        callbacks=[pruning_callback]
    )

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)

    log_loss = sklearn.metrics.log_loss(valid_y, preds)
    return log_loss

In [170]:
# Лучше подбирать гиперпараметры на всей data через кросс валидацию
import optuna.integration.lightgbm as lgb
def objective(trial):
    data = get_input("train.csv")
    data.drop(columns='id', inplace=True)
    mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
    data['target'] = data['target'].map(mapper)
    target = data['target']
    data = data.drop(columns=['target'])

    dtrain = lgb.Dataset(data, label=target)

    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        'num_class': 4,

        'max_depth': trial.suggest_int('max_depth', 3,7),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.03, 0.13 ),
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "multi_logloss")
    result = lgb.cv(
      params=param,
      train_set=dtrain,
      num_boost_round=10000,
      early_stopping_rounds=100,
      #verbose_eval=10,
      stratified=True,
      seed=42,
      metrics="multi_logloss",
      shuffle=True,
      nfold=3
    )
 
    log_loss = result['multi_logloss-mean'][-1] + result['multi_logloss-stdv'][-1]
    return log_loss

In [171]:
study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=20), direction="minimize"
    )
study.optimize(objective, n_trials=200)

print("Best trial:")
trial = study.best_trial

print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

[I 2021-05-25 20:05:15,460] A new study created in memory with name: no-name-67bb9b06-6056-4c5f-9440-69bfd5220d67


train.csv: shape = 100000 rows, 52 cols


[I 2021-05-25 20:06:07,912] Trial 0 finished with value: 1.095420663265998 and parameters: {'lambda_l1': 0.00024587192153438774, 'lambda_l2': 2.9664723207282242e-08, 'num_leaves': 223, 'feature_fraction': 0.7099892913219672, 'bagging_fraction': 0.7228809986014417, 'bagging_freq': 6, 'min_child_samples': 97, 'max_depth': 5, 'learning_rate': 0.08549446305776937}. Best is trial 0 with value: 1.095420663265998.


train.csv: shape = 100000 rows, 52 cols


KeyboardInterrupt: ignored

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
print(study.best_params)

{'lambda_l1': 1.45120566383297e-06, 'lambda_l2': 0.003368552965821498, 'num_leaves': 32, 'feature_fraction': 0.4471131328810426, 'bagging_fraction': 0.5888198372400193, 'bagging_freq': 4, 'min_child_samples': 32}


In [ ]:
study.best_value

1.0833801288748757

In [15]:
data = get_input("train.csv")
data.drop(columns='id', inplace=True)
mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
data['target'] = data['target'].map(mapper)
target = data['target']
data = data.drop(columns=['target'])

lgb_params = {
    "boosting_type ": "gbdt",
    "objective": "multiclass",
    "metric": "multi_logloss",
    #'num_boost_round': 100,
    'num_class': 4,
    'lambda_l1': 0.02259438646302076,
    'lambda_l2': 3.3834082296901886e-05,
    'num_leaves': 256,
    'feature_fraction': 0.42939073275678896,
    'bagging_fraction': 0.8760623722003144,
    'bagging_freq': 4,
    'min_child_samples': 71,
    'max_depth': 4,
    'learning_rate': 0.07658957460133804,
}

dtrain = lgb.Dataset(data=data, label=target)

result = lgb.cv(
    params=lgb_params,
    train_set=dtrain,
    num_boost_round=2000,
    early_stopping_rounds=100,
    verbose_eval=10,
    stratified=True,
    seed=42,
    metrics="multi_logloss",
    shuffle=True,
    nfold=5
)

train.csv: shape = 100000 rows, 52 cols
[10]	cv_agg's multi_logloss: 1.11339 + 0.00019763
[20]	cv_agg's multi_logloss: 1.10998 + 0.000341567
[30]	cv_agg's multi_logloss: 1.10748 + 0.000399434
[40]	cv_agg's multi_logloss: 1.1054 + 0.000482044
[50]	cv_agg's multi_logloss: 1.10365 + 0.000591791
[60]	cv_agg's multi_logloss: 1.10218 + 0.000701451
[70]	cv_agg's multi_logloss: 1.10096 + 0.000759168
[80]	cv_agg's multi_logloss: 1.09988 + 0.000727296
[90]	cv_agg's multi_logloss: 1.09888 + 0.000727316
[100]	cv_agg's multi_logloss: 1.09803 + 0.000788178
[110]	cv_agg's multi_logloss: 1.09726 + 0.00084155
[120]	cv_agg's multi_logloss: 1.09667 + 0.000892679
[130]	cv_agg's multi_logloss: 1.09616 + 0.000970538
[140]	cv_agg's multi_logloss: 1.09569 + 0.00102404
[150]	cv_agg's multi_logloss: 1.09526 + 0.00106349
[160]	cv_agg's multi_logloss: 1.09494 + 0.0011131
[170]	cv_agg's multi_logloss: 1.09459 + 0.00114867
[180]	cv_agg's multi_logloss: 1.09433 + 0.00113994
[190]	cv_agg's multi_logloss: 1.09407 + 0.

In [19]:
len(result['multi_logloss-mean'])

334

In [9]:
import lightgbm as lgb
data = get_input("train.csv")
test = get_input("test.csv")

data.drop(columns='id', inplace=True)
mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
data['target'] = data['target'].map(mapper)

target = data['target']
data = data.drop(columns=['target'])

#train, valid = train_test_split(
#    data, train_size=0.7, shuffle=True, random_state=1,
#)

#train.reset_index(inplace=True, drop=True)
#valid.reset_index(inplace=True, drop=True)

#y_train = train['target']
#x_train = train.drop(columns=['target'])
#y_valid = valid['target']
#x_valid = valid.drop(columns=['target'])

train.csv: shape = 100000 rows, 52 cols
test.csv: shape = 50000 rows, 51 cols


In [ ]:
lgb_params = {
    "boosting_type ": "gbdt",
    "objective": "multiclass",
    "metric": "multi_logloss",
    'n_estimators': 500,
    'num_class': 4,
    'lambda_l1': 0.02259438646302076,
    'lambda_l2': 3.3834082296901886e-05,
    'num_leaves': 256,
    'feature_fraction': 0.42939073275678896,
    'bagging_fraction': 0.8760623722003144,
    'bagging_freq': 4,
    'min_child_samples': 71,
    'max_depth': 3,
    'learning_rate': 0.07658957460133804,
    'random_state': 42,
}

cv = StratifiedKFold(n_splits=10, random_state=435, shuffle=True)

lgb_estimators, lgb_oof = lightgbm_cv_fit(
    lgb_params, data, target, cv, #categorical=categorical_feature_names
)

Tue May 25 22:41:19 2021, Cross-Validation, 100000 rows, 50 cols
Fold 1, Valid score = 1.09457
Fold 2, Valid score = 1.08862
Fold 3, Valid score = 1.09364
Fold 4, Valid score = 1.09471
Fold 5, Valid score = 1.09327
Fold 6, Valid score = 1.09222
Fold 7, Valid score = 1.09183
Fold 8, Valid score = 1.09551


In [78]:
# oof 1.0926 должен быть
print(f"Out of fold log loss {metrics.log_loss(target, lgb_oof)}")

Out of fold log loss 1.0945776931062208


In [44]:
lgb_estimators[0]

LGBMClassifier(bagging_fraction=0.8760623722003144, bagging_freq=4,
               boosting_type ='gbdt', feature_fraction=0.42939073275678896,
               lambda_l1=0.02259438646302076, lambda_l2=3.3834082296901886e-05,
               learning_rate=0.07658957460133804, max_depth=4,
               metric='multi_logloss', min_child_samples=71, n_estimators=334,
               num_class=4, num_leaves=256, objective='multiclass',
               random_state=42)

### Catboost

In [172]:
# optuna с кросс валидацией работает очень долго поэтому первое приближение получим 
# на отложенной выборке а потом дотюним
def objective(trial):
    data = get_input("train.csv")
    data.drop(columns='id', inplace=True)
    mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
    data['target'] = data['target'].map(mapper)
    target = data['target']
    data = data.drop(columns=['target'])

    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.35)

    param = {
        "loss_function": "MultiClass",
        "eval_metric": "MultiClass",
        "task_type": "GPU",
        
        #"colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        #"subsample": trial.suggest_float("subsample", 0.1, 1),

        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'depth': trial.suggest_int('depth', 3,7),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.13 ),
        'random_strength' :trial.suggest_int('random_strength', 0, 100),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-8, 100),                       
        #'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00), # for bayesian bootstrap only

        #'bootstrap_type': 'Bernoulli',
        #'leaf_estimation_method': 'Newton',

        'grow_policy': "SymmetricTree",
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "multi_logloss")
    gbm = cb.CatBoostClassifier(**param)

    gbm.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=100)
    preds = gbm.predict_proba(valid_x)


    log_loss = sklearn.metrics.log_loss(valid_y, preds)
    return log_loss

In [173]:
study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=20), direction="minimize"
    )
study.optimize(objective, n_trials=200)

print("Best trial:")
trial = study.best_trial

print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

[I 2021-05-25 20:07:18,768] A new study created in memory with name: no-name-234ea1e3-17bb-4364-985a-ec4112338ece


train.csv: shape = 100000 rows, 52 cols


[I 2021-05-25 20:07:19,978] Trial 0 finished with value: 1.186633703881519 and parameters: {'min_data_in_leaf': 24, 'depth': 3, 'learning_rate': 0.12078519555155853, 'random_strength': 58, 'l2_leaf_reg': 2.6581884406103866e-06}. Best is trial 0 with value: 1.186633703881519.


train.csv: shape = 100000 rows, 52 cols


[I 2021-05-25 20:07:25,654] Trial 1 finished with value: 1.0898680635762144 and parameters: {'min_data_in_leaf': 33, 'depth': 4, 'learning_rate': 0.07195112205603399, 'random_strength': 86, 'l2_leaf_reg': 1.0362623676141234}. Best is trial 1 with value: 1.0898680635762144.


...

train.csv: shape = 100000 rows, 52 cols


[I 2021-05-25 20:24:06,281] Trial 198 finished with value: 1.1032557383383232 and parameters: {'min_data_in_leaf': 80, 'depth': 5, 'learning_rate': 0.06550125070387704, 'random_strength': 83, 'l2_leaf_reg': 7.8067576913576975}. Best is trial 152 with value: 1.0836613774733865.


train.csv: shape = 100000 rows, 52 cols


[I 2021-05-25 20:24:13,011] Trial 199 finished with value: 1.0937628032274382 and parameters: {'min_data_in_leaf': 48, 'depth': 5, 'learning_rate': 0.03715741624813227, 'random_strength': 52, 'l2_leaf_reg': 3.6429876663953307}. Best is trial 152 with value: 1.0836613774733865.


Best trial:
  Params: 
    min_data_in_leaf: 43
    depth: 5
    learning_rate: 0.07928546561861827
    random_strength: 66
    l2_leaf_reg: 2.9448952422509165


In [174]:
optuna.visualization.plot_param_importances(study)

In [ ]:
data = get_input("train.csv")
data.drop(columns='id', inplace=True)
mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
data['target'] = data['target'].map(mapper)
target = data['target']
data = data.drop(columns=['target'])

cb_params = {
        "loss_function": "MultiClass",
        "eval_metric": "MultiClass",
        "task_type": "GPU",
        'min_data_in_leaf': 25,
        'depth': 3,
        'learning_rate': 0.09,
        'random_strength' :10,
        "l2_leaf_reg": 100,                       
        'grow_policy': "SymmetricTree",
        #"thread_count": 10,
        "random_seed": 27,
    }
cv_result = cb.cv(
        pool = cb.Pool(data, target),
        params = cb_params,
        plot=True,
        shuffle = True,
        stratified = True,
        seed = 42,
        iterations = 10000,
        early_stopping_rounds = 100,
        fold_count = 5,
        as_pandas = True,
        verbose_eval = 10
        )

In [56]:
cb_params = {
    "n_estimators": 1700,
    "loss_function": "MultiClass",
    "eval_metric": "MultiClass",
    "task_type": "GPU",
    'min_data_in_leaf': 25,
    'depth': 3,
    'learning_rate': 0.09,
    'random_strength' : 10,
    "l2_leaf_reg": 100,                       
    'grow_policy': "SymmetricTree",
    "random_seed": 27,
}

cv = StratifiedKFold(n_splits=10, random_state=435, shuffle=True)

cb_estimators, cb_oof = catboost_cv_fit(
    cb_params, data, target, cv, #categorical=categorical_feature_names
)

Tue May 25 22:15:34 2021, Cross-Validation, 100000 rows, 50 cols


0:	learn: 1.3358071	total: 6.86ms	remaining: 11.7s
10:	learn: 1.1503962	total: 59.3ms	remaining: 9.11s
20:	learn: 1.1186152	total: 115ms	remaining: 9.2s
30:	learn: 1.1109516	total: 168ms	remaining: 9.05s
40:	learn: 1.1084460	total: 218ms	remaining: 8.82s
50:	learn: 1.1068249	total: 279ms	remaining: 9.03s
60:	learn: 1.1054835	total: 326ms	remaining: 8.77s
70:	learn: 1.1044382	total: 378ms	remaining: 8.66s
80:	learn: 1.1031675	total: 430ms	remaining: 8.59s
90:	learn: 1.1020858	total: 478ms	remaining: 8.46s
100:	learn: 1.1010773	total: 527ms	remaining: 8.34s
110:	learn: 1.1001637	total: 583ms	remaining: 8.35s
120:	learn: 1.0994507	total: 637ms	remaining: 8.31s
130:	learn: 1.0986476	total: 683ms	remaining: 8.18s
140:	learn: 1.0978980	total: 729ms	remaining: 8.06s
150:	learn: 1.0972795	total: 775ms	remaining: 7.95s
160:	learn: 1.0966055	total: 820ms	remaining: 7.84s
170:	learn: 1.0959941	total: 870ms	remaining: 7.78s
180:	learn: 1.0954622	total: 921ms	remaining: 7.73s
190:	learn: 1.0949210	

0:	learn: 1.3358464	total: 7.98ms	remaining: 13.6s
10:	learn: 1.1505475	total: 45.3ms	remaining: 6.96s
20:	learn: 1.1189998	total: 81.4ms	remaining: 6.5s
30:	learn: 1.1114351	total: 121ms	remaining: 6.54s
40:	learn: 1.1090578	total: 168ms	remaining: 6.81s
50:	learn: 1.1070338	total: 205ms	remaining: 6.63s
60:	learn: 1.1061092	total: 237ms	remaining: 6.37s
70:	learn: 1.1050819	total: 279ms	remaining: 6.39s
80:	learn: 1.1038713	total: 315ms	remaining: 6.3s
90:	learn: 1.1027502	total: 351ms	remaining: 6.2s
100:	learn: 1.1016620	total: 387ms	remaining: 6.12s
110:	learn: 1.1009668	total: 435ms	remaining: 6.22s
120:	learn: 1.1001403	total: 468ms	remaining: 6.11s
130:	learn: 1.0992170	total: 506ms	remaining: 6.06s
140:	learn: 1.0986326	total: 552ms	remaining: 6.1s
150:	learn: 1.0979552	total: 589ms	remaining: 6.04s
160:	learn: 1.0972758	total: 622ms	remaining: 5.95s
170:	learn: 1.0967531	total: 659ms	remaining: 5.89s
180:	learn: 1.0962163	total: 693ms	remaining: 5.81s
190:	learn: 1.0957916	to

0:	learn: 1.3358516	total: 4.7ms	remaining: 7.98s
10:	learn: 1.1505582	total: 38.9ms	remaining: 5.98s
20:	learn: 1.1185105	total: 81ms	remaining: 6.48s
30:	learn: 1.1109069	total: 133ms	remaining: 7.18s
40:	learn: 1.1084489	total: 164ms	remaining: 6.64s
50:	learn: 1.1067498	total: 198ms	remaining: 6.4s
60:	learn: 1.1056608	total: 232ms	remaining: 6.22s
70:	learn: 1.1046609	total: 263ms	remaining: 6.03s
80:	learn: 1.1034057	total: 297ms	remaining: 5.93s
90:	learn: 1.1022852	total: 336ms	remaining: 5.94s
100:	learn: 1.1012795	total: 370ms	remaining: 5.86s
110:	learn: 1.1004068	total: 402ms	remaining: 5.76s
120:	learn: 1.0995968	total: 440ms	remaining: 5.75s
130:	learn: 1.0989039	total: 484ms	remaining: 5.8s
140:	learn: 1.0983122	total: 517ms	remaining: 5.72s
150:	learn: 1.0976526	total: 551ms	remaining: 5.65s
160:	learn: 1.0970506	total: 584ms	remaining: 5.58s
170:	learn: 1.0964905	total: 618ms	remaining: 5.53s
180:	learn: 1.0959484	total: 660ms	remaining: 5.54s
190:	learn: 1.0954418	tot

0:	learn: 1.3358415	total: 4.86ms	remaining: 8.26s
10:	learn: 1.1504352	total: 40.1ms	remaining: 6.16s
20:	learn: 1.1184555	total: 74.3ms	remaining: 5.94s
30:	learn: 1.1109127	total: 110ms	remaining: 5.91s
40:	learn: 1.1085115	total: 145ms	remaining: 5.88s
50:	learn: 1.1067563	total: 179ms	remaining: 5.77s
60:	learn: 1.1055743	total: 214ms	remaining: 5.75s
70:	learn: 1.1046333	total: 245ms	remaining: 5.62s
80:	learn: 1.1034868	total: 279ms	remaining: 5.57s
90:	learn: 1.1022985	total: 322ms	remaining: 5.69s
100:	learn: 1.1013109	total: 356ms	remaining: 5.63s
110:	learn: 1.1004479	total: 389ms	remaining: 5.57s
120:	learn: 1.0996436	total: 428ms	remaining: 5.58s
130:	learn: 1.0989112	total: 461ms	remaining: 5.52s
140:	learn: 1.0982411	total: 496ms	remaining: 5.48s
150:	learn: 1.0975838	total: 545ms	remaining: 5.59s
160:	learn: 1.0969933	total: 579ms	remaining: 5.53s
170:	learn: 1.0963747	total: 613ms	remaining: 5.48s
180:	learn: 1.0958984	total: 654ms	remaining: 5.49s
190:	learn: 1.095451

...

0:	learn: 1.3358560	total: 4.55ms	remaining: 7.73s
10:	learn: 1.1506490	total: 38.3ms	remaining: 5.89s
20:	learn: 1.1187239	total: 72.5ms	remaining: 5.8s
30:	learn: 1.1110360	total: 115ms	remaining: 6.2s
40:	learn: 1.1085911	total: 154ms	remaining: 6.21s
50:	learn: 1.1067379	total: 191ms	remaining: 6.17s
60:	learn: 1.1054622	total: 226ms	remaining: 6.08s
70:	learn: 1.1044918	total: 266ms	remaining: 6.1s
80:	learn: 1.1033010	total: 300ms	remaining: 5.99s
90:	learn: 1.1023078	total: 334ms	remaining: 5.9s
100:	learn: 1.1013128	total: 367ms	remaining: 5.82s
110:	learn: 1.1005225	total: 401ms	remaining: 5.74s
120:	learn: 1.0997717	total: 449ms	remaining: 5.86s
130:	learn: 1.0989748	total: 485ms	remaining: 5.81s
140:	learn: 1.0983482	total: 518ms	remaining: 5.73s
150:	learn: 1.0977306	total: 552ms	remaining: 5.66s
160:	learn: 1.0971425	total: 586ms	remaining: 5.6s
170:	learn: 1.0965254	total: 619ms	remaining: 5.53s
180:	learn: 1.0959883	total: 665ms	remaining: 5.58s
190:	learn: 1.0954873	tot

0:	learn: 1.3353526	total: 5.07ms	remaining: 8.61s
10:	learn: 1.1503368	total: 41.2ms	remaining: 6.32s
20:	learn: 1.1185383	total: 79.5ms	remaining: 6.36s
30:	learn: 1.1109338	total: 116ms	remaining: 6.22s
40:	learn: 1.1085331	total: 153ms	remaining: 6.19s
50:	learn: 1.1069139	total: 194ms	remaining: 6.26s
60:	learn: 1.1056520	total: 233ms	remaining: 6.27s
70:	learn: 1.1046285	total: 266ms	remaining: 6.11s
80:	learn: 1.1032847	total: 309ms	remaining: 6.18s
90:	learn: 1.1022847	total: 344ms	remaining: 6.08s
100:	learn: 1.1013187	total: 380ms	remaining: 6.01s
110:	learn: 1.1005207	total: 413ms	remaining: 5.92s
120:	learn: 1.0997788	total: 454ms	remaining: 5.93s
130:	learn: 1.0989819	total: 492ms	remaining: 5.9s
140:	learn: 1.0982632	total: 532ms	remaining: 5.88s
150:	learn: 1.0975611	total: 568ms	remaining: 5.82s
160:	learn: 1.0970523	total: 603ms	remaining: 5.76s
170:	learn: 1.0963759	total: 639ms	remaining: 5.71s
180:	learn: 1.0959039	total: 680ms	remaining: 5.71s
190:	learn: 1.0953944

0:	learn: 1.3358337	total: 4.92ms	remaining: 8.36s
10:	learn: 1.1505024	total: 40.7ms	remaining: 6.24s
20:	learn: 1.1185602	total: 80.5ms	remaining: 6.43s
30:	learn: 1.1109921	total: 117ms	remaining: 6.27s
40:	learn: 1.1085191	total: 153ms	remaining: 6.18s
50:	learn: 1.1067834	total: 187ms	remaining: 6.06s
60:	learn: 1.1055674	total: 225ms	remaining: 6.05s
70:	learn: 1.1044235	total: 258ms	remaining: 5.91s
80:	learn: 1.1031793	total: 297ms	remaining: 5.94s
90:	learn: 1.1022174	total: 332ms	remaining: 5.87s
100:	learn: 1.1011620	total: 367ms	remaining: 5.81s
110:	learn: 1.1003388	total: 400ms	remaining: 5.73s
120:	learn: 1.0995326	total: 445ms	remaining: 5.81s
130:	learn: 1.0987475	total: 481ms	remaining: 5.76s
140:	learn: 1.0979748	total: 528ms	remaining: 5.83s
150:	learn: 1.0972657	total: 568ms	remaining: 5.83s
160:	learn: 1.0966934	total: 604ms	remaining: 5.77s
170:	learn: 1.0960633	total: 640ms	remaining: 5.72s
180:	learn: 1.0956063	total: 679ms	remaining: 5.7s
190:	learn: 1.0950396

0:	learn: 1.3358562	total: 4.96ms	remaining: 8.43s
10:	learn: 1.1505040	total: 45.7ms	remaining: 7.02s
20:	learn: 1.1187260	total: 85.4ms	remaining: 6.83s
30:	learn: 1.1111453	total: 121ms	remaining: 6.54s
40:	learn: 1.1087374	total: 160ms	remaining: 6.46s
50:	learn: 1.1069453	total: 195ms	remaining: 6.3s
60:	learn: 1.1056946	total: 234ms	remaining: 6.3s
70:	learn: 1.1046602	total: 268ms	remaining: 6.16s
80:	learn: 1.1034385	total: 307ms	remaining: 6.13s
90:	learn: 1.1024115	total: 346ms	remaining: 6.11s
100:	learn: 1.1014061	total: 383ms	remaining: 6.06s
110:	learn: 1.1005122	total: 429ms	remaining: 6.14s
120:	learn: 1.0996749	total: 472ms	remaining: 6.16s
130:	learn: 1.0987554	total: 509ms	remaining: 6.09s
140:	learn: 1.0981715	total: 558ms	remaining: 6.17s
150:	learn: 1.0975480	total: 595ms	remaining: 6.1s
160:	learn: 1.0969516	total: 636ms	remaining: 6.08s
170:	learn: 1.0963167	total: 680ms	remaining: 6.08s
180:	learn: 1.0957567	total: 717ms	remaining: 6.01s
190:	learn: 1.0953552	t

0:	learn: 1.3358286	total: 5ms	remaining: 8.49s
10:	learn: 1.1506747	total: 42ms	remaining: 6.44s
20:	learn: 1.1187562	total: 78.5ms	remaining: 6.28s
30:	learn: 1.1112219	total: 115ms	remaining: 6.21s
40:	learn: 1.1088722	total: 157ms	remaining: 6.36s
50:	learn: 1.1072372	total: 210ms	remaining: 6.79s
60:	learn: 1.1059785	total: 243ms	remaining: 6.54s
70:	learn: 1.1048158	total: 276ms	remaining: 6.34s
80:	learn: 1.1037423	total: 312ms	remaining: 6.24s
90:	learn: 1.1026319	total: 347ms	remaining: 6.14s
100:	learn: 1.1015499	total: 383ms	remaining: 6.07s
110:	learn: 1.1007236	total: 423ms	remaining: 6.05s
120:	learn: 1.1000973	total: 459ms	remaining: 5.99s
130:	learn: 1.0993014	total: 504ms	remaining: 6.04s
140:	learn: 1.0986122	total: 546ms	remaining: 6.03s
150:	learn: 1.0979716	total: 590ms	remaining: 6.05s
160:	learn: 1.0974434	total: 631ms	remaining: 6.03s
170:	learn: 1.0968089	total: 668ms	remaining: 5.97s
180:	learn: 1.0962737	total: 708ms	remaining: 5.94s
190:	learn: 1.0957763	tot

In [57]:
print(f"Out of fold log loss {metrics.log_loss(target, cb_oof)}")

Out of fold log loss 1.090747335890732


In [118]:
cb_estimators[0]

### XGBoost

In [17]:
def objective(trial):
    data = get_input("train.csv")
    data.drop(columns='id', inplace=True)
    mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
    data['target'] = data['target'].map(mapper)
    target = data['target']
    data = data.drop(columns=['target'])

    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.35)
    dtrain = xgb.Dataset(train_x, label=train_y)
    dvalid = xgb.Dataset(valid_x, label=valid_y)

    param = {
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        "verbosity": -1,
        'num_class': 4,
        'max_depth': trial.suggest_int('max_depth', 3,7),
        'learning_rate': trial.suggest_uniform('learning_rate', 0, 0.13 ),
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "multi_logloss")
    gbm = lgb.train(
        param, dtrain, valid_sets=[dvalid], verbose_eval=False, callbacks=[pruning_callback]
    )

    preds = gbm.predict(valid_x)
    log_loss = sklearn.metrics.log_loss(valid_y, preds)
    return log_loss

In [ ]:
study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=20), direction="minimize"
    )
study.optimize(objective, n_trials=200)

print("Best trial:")
trial = study.best_trial

print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

In [ ]:
dtrain = xgb.DMatrix(
    data=data, label=target
)

xgb_params = {
    "booster": "gbtree",
    "eta": "0.1",
    "max_depth": 3,
    "random_seed": 42,
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',
    'random_state': 13,
    'learning_rate': 0.08356451010151393, 
    'gamma': 0.02911685058980812, 
    'max_depth': 3, 
    'min_child_weight': 10.748514454096288, 
    'max_delta_step': 2.4474818433727927, 
    'subsample': 0.6445037550866027, 
    'colsample_bytree': 0.07634753656242108, 
    'lambda': 13.663280761461781, 
    'alpha': 21.521205761694137, 
    'max_leaves': 48,
    'num_class': 4,
    'tree_method': 'gpu_hist',
}

cv_result_xgb = xgb.cv(
    params=xgb_params,
    dtrain=dtrain,
    num_boost_round=2000,
    #categorical_feature=categorical_feature_names,
    early_stopping_rounds=100,
    verbose_eval=10,
    stratified=True,
    seed=42,
    metrics="mlogloss",
    shuffle=True,
    nfold=5,
    #maximaze=True
)

In [43]:
cv_result_xgb.shape[0]

1838

In [58]:
xgb_params = {
    "booster": "gbtree",
    #"eta": 0.1,
    "random_seed": 42,
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',
    'random_state': 13,
    'learning_rate': 0.08356451010151393, 
    'gamma': 0.02911685058980812, 
    'max_depth': 3,
    'min_child_weight': 10.748514454096288, 
    'max_delta_step': 2.4474818433727927,
    'subsample': 0.6445037550866027,
    'colsample_bytree': 0.07634753656242108,
    'lambda': 13.663280761461781, 
    'alpha': 21.521205761694137, 
    'num_class': 4,
    'n_estimators': 700,  # надо потюнить
    'tree_method': 'gpu_hist',
}

In [59]:
cv = StratifiedKFold(n_splits=10, random_state=435, shuffle=True)

xgb_estimators, xgb_encoders, xgb_oof = xgboost_cv_fit(
    xgb_params, data, target, cv, #categorical=categorical_feature_names
)

Tue May 25 22:17:09 2021, Cross-Validation, 100000 rows, 50 cols
Fold 1, Valid score = 1.09345
Fold 2, Valid score = 1.08728
Fold 3, Valid score = 1.09217
Fold 4, Valid score = 1.09297
Fold 5, Valid score = 1.09204
Fold 6, Valid score = 1.08979
Fold 7, Valid score = 1.09069
Fold 8, Valid score = 1.09386
Fold 9, Valid score = 1.09052
Fold 10, Valid score = 1.08872


In [60]:
print(f"Out of fold log loss {metrics.log_loss(target, xgb_oof)}")

Out of fold log loss 1.0911490977954974


In [217]:
xgb_estimators[0]

XGBClassifier(alpha=21.521205761694137, colsample_bytree=0.07634753656242108,
              eta=0.1, eval_metric='mlogloss', gamma=0.02911685058980812,
              lambda=13.663280761461781, learning_rate=0.08356451010151393,
              max_delta_step=2.4474818433727927, max_leaves=48,
              min_child_weight=10.748514454096288, n_estimators=700,
              num_class=4, objective='multi:softprob', random_seed=42,
              random_state=13, subsample=0.6445037550866027,
              tree_method='gpu_hist')

### Модель второго уровня - логистическая регрессия

In [61]:
result_lgbm_df = pd.DataFrame(lgb_oof)
result_xgb_df = pd.DataFrame(xgb_oof)
result_cb_df = pd.DataFrame(cb_oof)
result_lgbm_df.rename(columns={0:'lgb_class_1',	1:'lgb_class_2',	2:'lgb_class_3',	3:'lgb_class_4'}, inplace=True)
result_xgb_df.rename(columns={0:'xgb_class_1',	1:'xgb_class_2',	2:'xgb_class_3',	3:'xgb_class_4'}, inplace=True)
result_cb_df.rename(columns={0:'cb_class_1',	1:'cb_class_2',	2:'cb_class_3',	3:'cb_class_4'}, inplace=True)
result_valid = pd.concat([result_lgbm_df, result_xgb_df, result_cb_df], axis=1) 

In [62]:
x_train_logreg, x_valid_logreg, y_train_logreg, y_valid_logreg = train_test_split(
    result_valid, target, train_size=0.7, shuffle=True, random_state=30,
)

params = {'random_state': 0, 
          'max_iter': 300, 
          'multi_class': 'multinomial', 
          'solver': 'lbfgs'}

logreg = LogisticRegression(**params)
logreg.fit(x_train_logreg, y_train_logreg)

# 1.08521
y_pred_logreg = logreg.predict_proba(x_valid_logreg)
metrics.log_loss(y_valid_logreg, y_pred_logreg)

1.0855461236990565

In [63]:
# Обучаем на всем наборе 1.08522
logreg = LogisticRegression(**params)
logreg.fit(result_valid, target)

LogisticRegression(max_iter=300, multi_class='multinomial', random_state=0)

### Модель второго уровня - ансамбль логистических регрессий

In [75]:
logreg_params = {'random_state': 0, 
          'max_iter': 500, 
          'multi_class': 'multinomial', 
          'solver': 'lbfgs',
          'C':1
          }

cv = StratifiedKFold(n_splits=7, random_state=435, shuffle=True)
estimators_log_reg, oof_log_reg, scalers = logistic_cv_fit(logreg_params, result_valid, target, cv)

Tue May 25 22:33:22 2021, Cross-Validation, 100000 rows, 12 cols
Fold 1, Valid score = 1.09015
Fold 2, Valid score = 1.08871
Fold 3, Valid score = 1.09197
Fold 4, Valid score = 1.08691
Fold 5, Valid score = 1.09126
Fold 6, Valid score = 1.09077
Fold 7, Valid score = 1.08693
Score by each fold: [1.09015, 1.08871, 1.09197, 1.08691, 1.09126, 1.09077, 1.08693]


In [76]:
# 1.08522
print(f"Out of fold log loss {metrics.log_loss(target, oof_log_reg)}")

Out of fold log loss 1.089527397991803


## Получение результата (одна модель логистической регрессии)

In [64]:
result_xgb_test = multi_estimators_predict(xgb_estimators, test.drop(columns='id'))
result_lgb_test = multi_estimators_predict(lgb_estimators, test.drop(columns='id'))
result_cb_test = multi_estimators_predict(cb_estimators, test.drop(columns='id'))

result_lgb_test_df = pd.DataFrame(result_lgb_test)
result_xgb_test_df = pd.DataFrame(result_xgb_test)
result_cb_test_df = pd.DataFrame(result_cb_test)
result_lgb_test_df.rename(columns={0:'lgb_class_1',	1:'lgb_class_2',	2:'lgb_class_3',	3:'lgb_class_4'}, inplace=True)
result_xgb_test_df.rename(columns={0:'xgb_class_1',	1:'xgb_class_2',	2:'xgb_class_3',	3:'xgb_class_4'}, inplace=True)
result_cb_test_df.rename(columns={0:'cb_class_1',	1:'cb_class_2',	2:'cb_class_3',	3:'cb_class_4'}, inplace=True)
result_for_logreg = pd.concat([result_lgb_test_df, result_xgb_test_df, result_cb_test_df], axis=1) 

In [65]:
y_pred_logreg = logreg.predict_proba(result_for_logreg)

In [66]:
test_pred_df = pd.DataFrame(y_pred_logreg)
test_pred_df.rename(columns={0:'class_1',	1:'class_2',	2:'class_3',	3:'class_4'}, inplace=True)
result = test.drop(columns = test.columns.to_list()[1:])
result = pd.concat([result, test_pred_df], axis=1)
result.to_csv('result.csv', index=False)

## Получение результата (ансамбль логистических регрессий)

In [78]:
result_xgb_test = multi_estimators_predict(xgb_estimators, test.drop(columns='id'))
result_lgb_test = multi_estimators_predict(lgb_estimators, test.drop(columns='id'))
result_cb_test = multi_estimators_predict(cb_estimators, test.drop(columns='id'))

result_lgb_test_df = pd.DataFrame(result_lgb_test)
result_xgb_test_df = pd.DataFrame(result_xgb_test)
result_cb_test_df = pd.DataFrame(result_cb_test)
result_lgb_test_df.rename(columns={0:'lgb_class_1',	1:'lgb_class_2',	2:'lgb_class_3',	3:'lgb_class_4'}, inplace=True)
result_xgb_test_df.rename(columns={0:'xgb_class_1',	1:'xgb_class_2',	2:'xgb_class_3',	3:'xgb_class_4'}, inplace=True)
result_cb_test_df.rename(columns={0:'cb_class_1',	1:'cb_class_2',	2:'cb_class_3',	3:'cb_class_4'}, inplace=True)
result_for_logreg = pd.concat([result_lgb_test_df, result_xgb_test_df, result_cb_test_df], axis=1) 

In [80]:
result = multi_estimators_predict(estimators_log_reg, result_for_logreg)

In [82]:
test_pred_df = pd.DataFrame(result)
test_pred_df.rename(columns={0:'class_1',	1:'class_2',	2:'class_3',	3:'class_4'}, inplace=True)
result = test.drop(columns = test.columns.to_list()[1:])
result = pd.concat([result, test_pred_df], axis=1)
result.to_csv('result.csv', index=False)